# Achitecture 1 : "Learning Image Matching by Simply Watching Video"

In [1]:
import numpy as np
import tensorflow as tf
import os
import time
import cv2
import sys

print(sys.version)

3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


## Parameters

In [2]:
_conv_layers = [6,96,96,128,128,128]
_activation = 'relu'

_batch_size = 8 #16 in the paper but too expensive for the GPU
_learning_rate = 1e-3
_epochs = 20
_step_test = 10
_step_viz = 50

_dataset = "KITTI"
# _dataset = "SINTEL"

if _dataset == "KITTI":
    _h,_w = (128,384)
if _dataset == "SINTEL":
    _h,_w = (128,256)

_data_folder = "data/%s"%_dataset
_train_folder = "train/%s"%_dataset
_previously_trained = True

if not os.path.exists(_train_folder):
    os.makedirs(_train_folder)
    print("Directory created :", _train_folder)

In [3]:
def activation(x,name=None):
    if _activation == 'sigmoid':
        return tf.nn.sigmoid(x,name)
    if _activation == 'relu':
        return tf.nn.relu(x,name)
    
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weights')

def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='bias')

## Create training batch

In [15]:
def generate_batch(dataset,batch_size, train = 1, quality ="clean"):
    
    if dataset == "SINTEL":
        frames = np.ndarray([3,batch_size,128,256, 3],np.float32)
        if train:
            folder = "train"
        else:
            folder = "test"
        
        seq = np.random.choice(os.listdir("data/"+dataset+"/"+folder+"/"+quality),batch_size)        
        for i in range(batch_size):
            index = np.random.randint(1,len(os.listdir("data/"+dataset+"/"+folder+"/"+quality+"/"+seq[i]))-2)            
            frames[0,i,:,:,:] = cv2.imread("data/"+dataset+"/"+folder+"/"+quality+"/"+seq[i]+"/frame_%s"%'{0:04}'.format(index)+".png")
            frames[1,i,:,:,:] = cv2.imread("data/"+dataset+"/"+folder+"/"+quality+"/"+seq[i]+"/frame_%s"%'{0:04}'.format(index+1)+".png")
            frames[2,i,:,:,:] = cv2.imread("data/"+dataset+"/"+folder+"/"+quality+"/"+seq[i]+"/frame_%s"%'{0:04}'.format(index+2)+".png")
    
    if dataset == "KITTI":
        frames = np.ndarray([3,batch_size,128,384, 3],np.float32)
        if train:
            folders = ["2011_09_26","2011_09_28","2011_09_30","2011_10_03"]
            folder = np.random.choice(folders,batch_size)
        else:
            folders = ["2011_09_29"]
            folder = np.random.choice(folders,batch_size)        
        
        for i in range(batch_size):
            seq = np.random.choice(os.listdir("data/"+dataset+"/kitti_resized/"+folder[i]))
            
            index = np.random.randint(1,len(os.listdir("data/"+dataset+"/kitti_resized/"+folder[i]+"/"+seq))-2)            
            frames[0,i,:,:,:] = cv2.imread("data/"+dataset+"/kitti_resized/"+folder[i]+"/"+seq+"/%s"%'{0:010}'.format(index)+".png")
            frames[1,i,:,:,:] = cv2.imread("data/"+dataset+"/kitti_resized/"+folder[i]+"/"+seq+"/%s"%'{0:010}'.format(index+1)+".png")
            frames[2,i,:,:,:] = cv2.imread("data/"+dataset+"/kitti_resized/"+folder[i]+"/"+seq+"/%s"%'{0:010}'.format(index+2)+".png")
    
    
    return np.concatenate([frames[0],frames[2]],axis=3)/255.,frames[1]/255.



## Convolutional Network graph

In [5]:
sess = tf.InteractiveSession()

x = tf.placeholder("float32",[None,_h,_w,6],name='x_input')
x_gt = tf.placeholder("float32",[None,_h,_w,3],name='x_ground-truth')

#CONVOLUTIONAL ENCODER

with tf.variable_scope("CONV-BLOCK1") as scope:
    with tf.variable_scope('conv1') as scope:
        W1_1 = weight_variable([3,3,_conv_layers[0],_conv_layers[1]])
        b1_1 = bias_variable([_conv_layers[1]])             
        preacti1_1 = tf.nn.bias_add(tf.nn.conv2d(x,W1_1,[1,1,1,1],padding='SAME'),b1_1)
        conv1_1 = activation(preacti1_1,name=scope.name)

    with tf.variable_scope('conv2') as scope:
        W2_1 = weight_variable([3,3,_conv_layers[1],_conv_layers[1]])
        b2_1 = bias_variable([_conv_layers[1]])             
        preacti2_1 = tf.nn.bias_add(tf.nn.conv2d(conv1_1,W2_1,[1,1,1,1],padding='SAME'),b2_1)
        conv2_1 = activation(preacti2_1,name=scope.name)

    with tf.variable_scope('conv3') as scope:
        W3_1 = weight_variable([3,3,_conv_layers[1],_conv_layers[1]])
        b3_1 = bias_variable([_conv_layers[1]])             
        preacti3_1 = tf.nn.bias_add(tf.nn.conv2d(conv2_1,W3_1,[1,1,1,1],padding='SAME'),b3_1)
        conv3_1 = activation(preacti3_1,name=scope.name)
        
    pool_1 = tf.nn.max_pool(conv3_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    

    
for i in range(2,6):    
    with tf.variable_scope("CONV-BLOCK%s"%i) as scope:
        with tf.variable_scope('conv1') as scope:
            globals()['W1_%s'%i] = weight_variable([3,3,_conv_layers[i-1],_conv_layers[i]])
            globals()['b1_%s'%i] = bias_variable([_conv_layers[i]])             
            globals()['preacti1_%s'%i] = tf.nn.bias_add(tf.nn.conv2d(globals()['pool_%s'%(i-1)],globals()['W1_%s'%i],[1,1,1,1],padding='SAME'),globals()['b1_%s'%i])
            globals()['conv1_%s'%i] = activation(globals()['preacti1_%s'%i],name=scope.name)

        with tf.variable_scope('conv2') as scope:
            globals()['W2_%s'%i] = weight_variable([3,3,_conv_layers[i],_conv_layers[i]])
            globals()['b2_%s'%i] = bias_variable([_conv_layers[i]])             
            globals()['preacti2_%s'%i] = tf.nn.bias_add(tf.nn.conv2d(globals()['conv1_%s'%i],globals()['W2_%s'%i],[1,1,1,1],padding='SAME'),globals()['b2_%s'%i])
            globals()['conv2_%s'%i] = activation(globals()['preacti2_%s'%i],name=scope.name)

        with tf.variable_scope('conv3') as scope:
            globals()['W3_%s'%i] = weight_variable([3,3,_conv_layers[i],_conv_layers[i]])
            globals()['b3_%s'%i] = bias_variable([_conv_layers[i]])             
            globals()['preacti3_%s'%i] = tf.nn.bias_add(tf.nn.conv2d(globals()['conv2_%s'%i],globals()['W3_%s'%i],[1,1,1,1],padding='SAME'),globals()['b3_%s'%i])
            globals()['conv3_%s'%i] = activation(globals()['preacti3_%s'%i],name=scope.name)

        globals()['pool_%s'%i] = tf.nn.max_pool(globals()['conv3_%s'%i], ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')

#DECONVOLUTIONAL DECODER

batch_size = tf.shape(x)[0]

with tf.variable_scope("DECONV-BLOCK5") as scope:
    with tf.variable_scope('deconv1') as scope:
        W1_d5 = weight_variable([4,4,_conv_layers[5],_conv_layers[4]])
        b1_d5 = bias_variable([_conv_layers[4]])
        convtr_d5 = tf.nn.conv2d_transpose(pool_5,W1_d5,[batch_size,int(_h/2**4),int(_w/2**4),_conv_layers[4]],[1,2,2,1],padding='SAME')
        preacti1_d5 = tf.nn.bias_add(convtr_d5,b1_d5)
        conv1_d5 = activation(preacti1_d5,name=scope.name)

    with tf.variable_scope('deconv2') as scope:
        W2_d5 = weight_variable([3,3,_conv_layers[4],_conv_layers[4]])
        b2_d5 = bias_variable([_conv_layers[4]])             
        preacti2_d5 = tf.nn.bias_add(tf.nn.conv2d(conv1_d5,W2_d5,[1,1,1,1],padding='SAME'),b2_d5)
        conv2_d5 = activation(preacti2_d5,name=scope.name)

    with tf.variable_scope('deconv3') as scope:
        W3_d5 = weight_variable([3,3,_conv_layers[4],_conv_layers[4]])
        b3_d5 = bias_variable([_conv_layers[4]])             
        preacti3_d5 = tf.nn.bias_add(tf.nn.conv2d(conv2_d5,W3_d5,[1,1,1,1],padding='SAME'),b3_d5)
        conv3_d5 = activation(preacti3_d5,name=scope.name)

        
for i in range(4,1,-1):
    with tf.variable_scope("DECONV-BLOCK%s"%i) as scope:
        
        globals()['concat_d%s'%i] = tf.concat(3,[globals()['conv3_d%s'%(i+1)],globals()['pool_%s'%(i)]])
        
        with tf.variable_scope('deconv1') as scope:
            globals()['W1_d%s'%i] = weight_variable([4,4,_conv_layers[i-1],2*_conv_layers[i]])
            globals()['b1_d%s'%i] = bias_variable([_conv_layers[i-1]])
            globals()['convtr_d%s'%i] = tf.nn.conv2d_transpose(globals()['concat_d%s'%i],globals()['W1_d%s'%i],[batch_size,int(_h/2**(i-1)),int(_w/2**(i-1)),_conv_layers[i-1]],[1,2,2,1],padding='SAME')
            globals()['preacti1_d%s'%i] = tf.nn.bias_add(globals()['convtr_d%s'%i],globals()['b1_d%s'%i])
            globals()['conv1_d%s'%i] = activation(globals()['preacti1_d%s'%i],name=scope.name)
            
        with tf.variable_scope('deconv2') as scope:
            globals()['W2_d%s'%i] = weight_variable([3,3,_conv_layers[i-1],_conv_layers[i-1]])
            globals()['b2_d%s'%i] = bias_variable([_conv_layers[i-1]])  
            globals()['preacti2_d%s'%i] = tf.nn.bias_add(tf.nn.conv2d(globals()['conv1_d%s'%i],globals()['W2_d%s'%i],[1,1,1,1],padding='SAME'),globals()['b2_d%s'%i])
            globals()['conv2_d%s'%i] = activation(globals()['preacti2_d%s'%i],name=scope.name)

        with tf.variable_scope('deconv3') as scope:
            globals()['W3_d%s'%i] = weight_variable([3,3,_conv_layers[i-1],_conv_layers[i-1]])
            globals()['b3_d%s'%i] = bias_variable([_conv_layers[i-1]])             
            globals()['preacti3_d%s'%i] = tf.nn.bias_add(tf.nn.conv2d(globals()['conv2_d%s'%i],globals()['W3_d%s'%i],[1,1,1,1],padding='SAME'),globals()['b3_d%s'%i])
            globals()['conv3_d%s'%i] = activation(globals()['preacti3_d%s'%i],name=scope.name)
            
            
with tf.variable_scope("DECONV-BLOCK1") as scope:
    with tf.variable_scope('deconv1') as scope:
        W1_d1 = weight_variable([4,4,3,_conv_layers[1]])
        b1_d1 = bias_variable([3])
        convtr_d1 = tf.nn.conv2d_transpose(conv3_d2,W1_d1,[batch_size,_h,_w,3],[1,2,2,1],padding='SAME')
        preacti1_d1 = tf.nn.bias_add(convtr_d1,b1_d1)
        conv1_d1 = activation(preacti1_d1,name=scope.name)

    with tf.variable_scope('deconv2') as scope:
        W2_d1 = weight_variable([3,3,3,3])
        b2_d1 = bias_variable([3])             
        preacti2_d1 = tf.nn.bias_add(tf.nn.conv2d(conv1_d1,W2_d1,[1,1,1,1],padding='SAME'),b2_d1)
        conv2_d1 = activation(preacti2_d1,name=scope.name)

    with tf.variable_scope('deconv3') as scope:
        W3_d1 = weight_variable([3,3,3,3])
        b3_d1 = bias_variable([3])             
        preacti3_d1 = tf.nn.bias_add(tf.nn.conv2d(conv2_d1,W3_d1,[1,1,1,1],padding='SAME'),b3_d1)
        conv3_d1 = activation(preacti3_d1,name=scope.name)


Potential improvements : add dropout and batch normalization (not in the paper)

## Loss and metrics

In [6]:
def charbonnier_loss(x):
    return tf.sqrt(0.1**2+tf.reduce_mean(tf.square(x)))
    
def interpol_error(x,gt):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.sub(x,gt))))

def normalized_interpol_error(x,gt):
    norm_grad = cv2.Sobel(gt,cv2.CV_64F,1,0,ksize=5)**2+cv2.Sobel(gt,cv2.CV_64F,0,1,ksize=5)**2 
    return tf.sqrt(tf.reduce_mean(tf.div(tf.square(tf.sub(x,gt)),(norm_grad+1))))

In [7]:
with tf.name_scope('loss') as scope:
    cost = charbonnier_loss(tf.sub(conv3_d1,x_gt))
    tf.summary.scalar('loss',cost)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(_learning_rate, global_step,10000, 0.99, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,global_step=global_step)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(_train_folder+'/cnn', sess.graph)
saver = tf.train.Saver()

#initialization
init = tf.global_variables_initializer()

#launch graph
sess.run(init)

# restore weights from model
# if os.path.exists(_train_folder+"/model.ckpt"):
#     saver.restore(sess, _train_folder+"/model.ckpt")
#     print("Model restored.")

## Visualization tools

In [8]:
test_frames = generate_batch(_dataset,8,train = 0)
cv2.imwrite(_train_folder+"/ground_truth.png",np.concatenate(255*test_frames[1],0))

def test(i):
    x_reconstruct =  sess.run(conv3_d1, feed_dict={x: test_frames[0], x_gt:test_frames[1]})   
    
    cv2.namedWindow('reconstruction',flags= cv2.WINDOW_NORMAL)
    cv2.imshow('reconstruction',np.concatenate(x_reconstruct,0))
        
    cv2.namedWindow('ground-truth',flags= cv2.WINDOW_NORMAL)
    cv2.imshow('ground-truth',np.concatenate(test_frames[1],0))
    
    cv2.imwrite(_train_folder+"/reconstr_%s"%'{0:04}'.format(int(i))+".png",255*np.concatenate(x_reconstruct,0))
            
    cv2.waitKey(5)

## Launch Training

In [ ]:
start = time.time()
c = np.inf
for i in range(100000):
    
    batch = generate_batch(_dataset,_batch_size)
    feed = {x: batch[0], x_gt:batch[1]}
    summary,_ = sess.run([merged,optimizer], feed_dict=feed)
    
    if i % _step_test == 0 and i>0:
        c_old = c
        c = sess.run(cost, feed_dict=feed)
        print("STEP %d: LOSS_BATCH = %f, TIME %f"%(i,c,time.time()-start))
        save_path = saver.save(sess, _train_folder+"/model.ckpt")
        writer.add_summary(summary, i)
    
    if i % _step_viz == 0 and i>0:
        test(i/_step_viz)
    
        
        
print("Optimization Finished")


STEP 10: LOSS_BATCH = 0.436255, TIME 20.175762
STEP 20: LOSS_BATCH = 0.449289, TIME 40.735403
STEP 30: LOSS_BATCH = 0.433329, TIME 60.672759
STEP 40: LOSS_BATCH = 0.427462, TIME 83.582505
STEP 50: LOSS_BATCH = 0.425972, TIME 105.310482
STEP 60: LOSS_BATCH = 0.403595, TIME 128.272421
STEP 70: LOSS_BATCH = 0.432473, TIME 148.707471
STEP 80: LOSS_BATCH = 0.428276, TIME 167.997010
